In [28]:
from google.cloud import pubsub_v1
from google.cloud import bigquery
import json

In [29]:
project_id='case-study-airflow'
topic_name = 'projects/{0}/topics/tweepy_topic_dummy'.format(project_id)
sub_name = 'tweepy_sub_dummy'
dataset = 'ds_dummy'
table = 'stock_data'

subscriber = pubsub_v1.SubscriberClient()
print(topic_name)

projects/case-study-airflow/topics/tweepy_topic_dummy


In [30]:
def to_bq_table(dataset, table, data):
    client = bigquery.Client()
    dataset_ref = client.dataset(dataset)
    table_ref = dataset_ref.table(table)
    table = client.get_table(table_ref)
    errors = client.insert_rows(table, data)
    if not errors:
        print('Loaded {} row(s) into {}:{}'.format(len(data), dataset, table))
    else:
        print(data)
        print('Errors:')
        for error in errors:
            print(error)
            
def collect(data):
    decoded_data = data.decode()
    json_data = json.loads(decoded_data)
    x = json_data.get('messages').replace('\n', '').split(',')
    data = [{'Date':x[0],
       'Open':x[1],
       'High':x[2],
       'Low':x[3],
       'Close':x[4],
       'Volume':x[5],
       'Adj_close':x[6]}]
    #print(to_load)
    to_bq_table(dataset, table, data)
    return

def collect2(data):
    decoded_data = data.decode()
    print("Decoded data:")
    print(decoded_data)
    json_data = json.loads(decoded_data)
    print('Json data')
    print(json_data)
    x = json_data.get('messages').split('\n')
    data = []
    print(x)
    for row in x[1:]:
        rd = row.split(',')
        data.append({'Date':rd[0],
          'Open':rd[1],
          'High':rd[2],
          'Low':rd[3],
          'Close':rd[4],
          'Volume':rd[5],
          'Adj_close':rd[6]})
    #print(to_load)
    to_bq_table(dataset, table, data)
    return

def subscriber_callback(message):
    #print("this will be processed")
    collect2(message.data)
    message.ack()
    
    
sub_path = subscriber.subscription_path(project_id, sub_name)
#resp = subscriber.pull(sub_path, max_messages=2)
sub = subscriber.subscribe(sub_path, subscriber_callback)

In [31]:
try:
    sub.result(timeout=10)
except Exception as e:
    raise
finally:
    sub.close


AttributeError: 'StreamingPullFuture' object has no attribute 'close'

Decoded data:
{"messages": "Date,Open,High,Low,Close,Volume,Adj Close\n2010-01-04,213.429998,214.499996,212.38000099999996,214.009998,123432400,27.727039\n2010-01-05,214.599998,215.589994,213.249994,214.379993,150476200,27.774976000000002\n2010-01-06,214.379993,215.23,210.750004,210.969995,138040000,27.333178000000004\n2010-01-07,211.75,212.000006,209.050005,210.58,119282800,27.28265\n2010-01-08,210.299994,212.000006,209.06000500000002,211.98000499999998,111902700,27.464034\n2010-01-11,212.79999700000002,213.000002,208.450005,210.11000299999998,115557400,27.221758"}
Json data
{'messages': 'Date,Open,High,Low,Close,Volume,Adj Close\n2010-01-04,213.429998,214.499996,212.38000099999996,214.009998,123432400,27.727039\n2010-01-05,214.599998,215.589994,213.249994,214.379993,150476200,27.774976000000002\n2010-01-06,214.379993,215.23,210.750004,210.969995,138040000,27.333178000000004\n2010-01-07,211.75,212.000006,209.050005,210.58,119282800,27.28265\n2010-01-08,210.299994,212.000006,209.060005

Loaded 6 row(s) into ds_dummy:Table(TableReference(DatasetReference('case-study-airflow', 'ds_dummy'), 'stock_data'))
Decoded data:
{"messages": "Date,Open,High,Low,Close,Volume,Adj Close\n2010-01-04,213.429998,214.499996,212.38000099999996,214.009998,123432400,27.727039\n2010-01-05,214.599998,215.589994,213.249994,214.379993,150476200,27.774976000000002\n2010-01-06,214.379993,215.23,210.750004,210.969995,138040000,27.333178000000004\n2010-01-07,211.75,212.000006,209.050005,210.58,119282800,27.28265\n2010-01-08,210.299994,212.000006,209.06000500000002,211.98000499999998,111902700,27.464034\n2010-01-11,212.79999700000002,213.000002,208.450005,210.11000299999998,115557400,27.221758"}
Json data
{'messages': 'Date,Open,High,Low,Close,Volume,Adj Close\n2010-01-04,213.429998,214.499996,212.38000099999996,214.009998,123432400,27.727039\n2010-01-05,214.599998,215.589994,213.249994,214.379993,150476200,27.774976000000002\n2010-01-06,214.379993,215.23,210.750004,210.969995,138040000,27.333178000

In [ ]:
a = "this is data".encode('utf-8')

In [ ]:
x = ['2010-01-04', '213.429998', '214.499996', '212.38000099999996', '214.009998', '123432400', '27.727039']

In [ ]:
to_bq_table(dataset, table, data)

In [56]:
files = ['ION_PROMO_20101225FW.DAT', 'ION_PROMO_20190808.DAT', 'ION_PROMO_20190903.DAT', 'ION_PROMO_20190904.DAT', 'ION_PROMO_20190905.DAT', 'ION_PROMO_20190906.DAT', 'ION_PROMO_20190907.DAT', 'ION_PROMO_20190908.DAT', 'ION_PROMO_20190909.DAT', 'ION_PROMO_20190910.DAT', 'ION_PROMO_20190911.DAT', 'ION_PROMO_20190912.DAT']

In [63]:
def validate_file_name(file, feed):
    if feed=='ION_PROMO':
        date = file.replace('.gz','').replace('.DAT','').split('_')[-1]
        return 'HF' not in file and 'FW' not in file and len(date)==8 and ''.join(date[-2:])!='00'
    elif feed in ['PRICE', 'DAILY_SCAN']:
        date = file.replace('.gz', '').replace('.DAT', '').split('_')[-2]
        return 'HF' not in file and 'FW' not in file and len(date) == 8 and ''.join(date[-2:]) != '00'

In [64]:
validate_file_name('ION_PROMO_20101225FW.DAT', 'ION_PROMO')

False

In [65]:
print([file for file in map(validate_file_name, files, 'ION_PROMO')])

[None, None]


In [66]:
files = [('ION_PROMO_20101225FW.DAT', 'ION_PROMO'), ('ION_PROMO_20190808.DAT','ION_PROMO')]

In [62]:
print([x for x in filter(validate_file_name, files)])

TypeError: validate_file_name() missing 1 required positional argument: 'feed'

In [67]:
p = [x for x in filter(lambda x:validate_file_name(x[0], x[1]), files)]

In [68]:
[x[0] for x in p]

['ION_PROMO_20190808.DAT']

In [57]:
[(file, 'ION_PROMO') for file in files]

[('ION_PROMO_20101225FW.DAT', 'ION_PROMO'),
 ('ION_PROMO_20190808.DAT', 'ION_PROMO'),
 ('ION_PROMO_20190903.DAT', 'ION_PROMO'),
 ('ION_PROMO_20190904.DAT', 'ION_PROMO'),
 ('ION_PROMO_20190905.DAT', 'ION_PROMO'),
 ('ION_PROMO_20190906.DAT', 'ION_PROMO'),
 ('ION_PROMO_20190907.DAT', 'ION_PROMO'),
 ('ION_PROMO_20190908.DAT', 'ION_PROMO'),
 ('ION_PROMO_20190909.DAT', 'ION_PROMO'),
 ('ION_PROMO_20190910.DAT', 'ION_PROMO'),
 ('ION_PROMO_20190911.DAT', 'ION_PROMO'),
 ('ION_PROMO_20190912.DAT', 'ION_PROMO')]